# <span style="color:purple">Script edit dot</span>

In [1635]:
import pydot
from pydot import Edge
from pydot import Node
import re
global subgraphs, parents
subgraphs={}
parents={}

## Our goal
We want to change the following graph in order to have new nodes to visualize the flow between functions
<img src="../Tests/test_plusieurs_appels_imbriqués/graphe.png" style="width:400px">

# Key functions 

#### Import a graph from dot and return the pydot graph

In [1636]:
def graph_from_dot(file):
    """
       Import a graph from dot and return the pydot graph
       
       :param file: path to the file to import
       :return: pydot graph
       :rtype: Dot
    """
    (graph, ) = pydot.graph_from_dot_file(str(file))
    return graph

In [1637]:
G = graph_from_dot("../Tests/while_with_foo/test.c.011t.cfg.dot");

#### Extract subgraphs, edges, nodes

In [1638]:
def extract_subgraphs(subgraph_list):
    global subgraphs, parents
    """
       Extract subgraphs recursively from a list of subgraphs, and the hierarchy of the subgraphs
       
       :param graph: graph or subgraph from which subgraphs are to be extracted
       :return: List of subgraphs
       :rtype: List
    """
    new_subgraphs = []
    for subgraph in subgraph_list:
        subgraph.set_name(subgraph.get_name().strip("\"").replace("cluster_", ""))
        subgraphs[subgraph.get_name()] = subgraph
        children_subgraphs = subgraph.get_subgraph_list() 
        new_subgraphs += children_subgraphs
        for child in children_subgraphs:
            parents[child.get_name().strip("\"").replace("cluster_", "")] = subgraph
    if(new_subgraphs == []):
        return
    else:
        return extract_subgraphs(new_subgraphs)

In [1639]:
extract_subgraphs([G])
print(subgraphs)

{'test.c.011t.cfg': <pydot.Dot object at 0x7f7e78655320>, 'foo': <pydot.Subgraph object at 0x7f7e7864fcf8>, 'main': <pydot.Subgraph object at 0x7f7e7864fd68>, '1_1': <pydot.Subgraph object at 0x7f7e786d8518>}


In [1640]:
def extract_nodes(graph):
    """
       Extract nodes from a graph or subgraph
       
       :param graph: graph or subgraph from which nodes are to be extracted
       :return: List of nodes
       :rtype: List
    """
    return graph.get_nodes()

In [1641]:
nodes_main = extract_nodes(subgraphs['main'])
print(nodes_main)

[<pydot.Node object at 0x7f7e786ba0b8>, <pydot.Node object at 0x7f7e787d8240>, <pydot.Node object at 0x7f7e786ba400>, <pydot.Node object at 0x7f7e786d1ac8>, <pydot.Node object at 0x7f7e786f4c88>]


In [1642]:
def extract_edges(graph):
    """
       Extract edges from a graph or subgraph
       
       :param graph: graph or subgraph from which edges are to be extracted
       :return: List of edges
       :rtype: List
    """
    return graph.get_edges()

In [1643]:
edges_main = extract_edges(subgraphs['main'])
for edge in edges_main:
    print(edge)

fn_1_basic_block_0:s -> fn_1_basic_block_2:n  [style="solid,bold", color=blue, weight=100, constraint=true, label="[0%]"];
fn_1_basic_block_2:s -> fn_1_basic_block_3:n  [style="solid,bold", color=blue, weight=100, constraint=true, label="[0%]"];
fn_1_basic_block_3:s -> fn_1_basic_block_4:n  [style="solid,bold", color=black, weight=10, constraint=true, label="[0%]"];
fn_1_basic_block_3:s -> fn_1_basic_block_5:n  [style="solid,bold", color=black, weight=10, constraint=true, label="[0%]"];
fn_1_basic_block_4:s -> fn_1_basic_block_6:n  [style="solid,bold", color=blue, weight=100, constraint=true, label="[0%]"];
fn_1_basic_block_5:s -> fn_1_basic_block_6:n  [style="solid,bold", color=blue, weight=100, constraint=true, label="[0%]"];
fn_1_basic_block_6:s -> fn_1_basic_block_3:n  [style="dotted,bold", color=blue, weight=10, constraint=false, label="[0%]"];
fn_1_basic_block_6:s -> fn_1_basic_block_7:n  [style="solid,bold", color=black, weight=10, constraint=true, label="[0%]"];
fn_1_basic_bloc

#### Add an edge, a node

In [1644]:
def add_edge(graph, tail_node, head_node, label="[0%]", style="solid, bold", color="black", constraint="true"):
    """
       Add an edge to a graph or subgraph
       
       :param graph: graph or subgraph to which the edge is added
       :param tail_node: origin of the edge
       :param head_node: destination of the edge
       :param label: the label
       :param style: the style
       :param color: the color
       :param constraint: the constraint
    """
    edge = Edge(tail_node, head_node, label=label, style=style, color=color, constraint="true")
    graph.add_edge(edge)
    return

In [1645]:
add_edge(subgraphs['main'], nodes_main[0].get_name()+":n", nodes_main[1].get_name()+":s", color="red")

In [1646]:
def add_node(graph, node_name, label, shape='record', style='filled', fillcolor='lightgrey'):
    """
       Add a node to a graph or subgraph
       
       :param graph: graph or subgraph to which the node is added
       :param node_name: name of the node
       :param label: label of the node
       :param shape: shape of the node (default "record")
       :param style: style of the node (default "field")
       :param fillcolor: color of the node (default "lightgrey")
       :return: the node created
       :rtype: Node
    """
    node = Node(name=node_name, shape=shape, style=style, fillcolor=fillcolor, label=label)
    graph.add_node(node)
    return node

In [1647]:
add_node(G, 'node_yolo', 'yolo')

# Create the new node

#### Analyse and create labels if a function call is found

In [1648]:
def analyse_label_function_calls(node):
    """
        Find if it exists, the first call to a function in a node
        
        :param node: the node examined
        :return: tuple containing the name of the function, the function call and its position in the label of the 
                 node
        :rtype: tuple
    """
    global subgraphs
    regex = r"(?:\|_\w+\\ \=\\ |\|)\w+\\\s\(\w*\)"
    label = node.get_attributes()['label']
    for match in re.finditer(regex, label, re.MULTILINE):
        function_call = re.search(r'\w+\\\s\(\w*\)', match.group()).group()
        function_name = re.search(r'\w+', function_call)
        if function_name is not None:
            if function_name.group() in subgraphs:
                return (function_name.group(), function_call, match.start())
    return (None, None, None)

In [1649]:
nodes = extract_nodes(subgraphs['1_1'])
node = nodes[0]
analyse = analyse_label_function_calls(node)
print(analyse)
print(node.get_attributes()['label'])

('foo', 'foo\\ (b)', 25)
"{ FREQ:0 |\<bb\ 3\>:\l\
|_1\ =\ foo\ (b);\l\
|if\ (_1\ \>\ 3)\l\
\ \ goto\ \<bb\ 4\>;\ [0.00%]\l\
else\l\
\ \ goto\ \<bb\ 5\>;\ [0.00%]\l\
}"


In [1650]:
def create_labels(node, function_call, function_call_line):
    """
        Create the labels for the new node, and for the node cut in half.
        
        :param node: the node cut
        :param function_call: the string containing the function call used as a regex expression
        :param function_call_line: the line where the function call is made
        :return: tuple containing the new label for the node cut, the label for the new node, and the value of
                 the cut node's bb
        :rtype: tuple
    """
    
    label_node = node.get_attributes()["label"]
    label_node_from_function_line = label_node[function_call_line:]
    regex = re.escape(function_call) + r';\\l\\\n'
    match_function = re.search(regex, label_node_from_function_line, re.S)
    
    bb = int(re.search(r'\d+', re.search(r'<bb\\\ \d+\\>', label_node).group()).group())
    
    label_node1 = label_node[0:function_call_line] + '|call\ ' + function_call + ';\l\\\n}"'
    
    match_variable = re.search(r'\w+\\ =\\ ' + re.escape(function_call), label_node_from_function_line)

    label_node2 = '"{ FREQ:0 |\<bb\ ' + str(bb) + '\>:\l\\\n|'
    
    if match_variable is not None:
        variable = re.search(r'\w+', match_variable.group()).group()
        label_node2 += variable + '\ =\ '
    label_node2 += 'return\ ' + function_call + ';\\l\\\n' + label_node_from_function_line[match_function.end():]
    
    return (label_node1, label_node2, int(bb))
    
    

In [1651]:
(label_node1, label_node2, bb) = create_labels(node, analyse[1], analyse[2])
print("---------")
print(label_node1)
print(label_node2)
print(bb)

---------
"{ FREQ:0 |\<bb\ 3\>:\l\
|call\ foo\ (b);\l\
}"
"{ FREQ:0 |\<bb\ 3\>:\l\
|_1\ =\ return\ foo\ (b);\l\
|if\ (_1\ \>\ 3)\l\
\ \ goto\ \<bb\ 4\>;\ [0.00%]\l\
else\l\
\ \ goto\ \<bb\ 5\>;\ [0.00%]\l\
}"
3


#### Update the value of the nodes: name and labels

In [1652]:
def update_node(node, attribute, value):
    """
        Update the node's attribute value.
        
        :param node: the node updated
        :param attribute: the attribute
        :param value: the value
    """
    node.set(attribute, value)
    return

In [1653]:
label_before = node.get_attributes()["label"]
update_node(node, "label", "new_label of the node");
print(node.get_attributes()["label"])
update_node(node, "label", label_before)

new_label of the node


In [1654]:
def update_bb_string(string, bb):
    """ 
        update recursivly the values of the bb found in the string. The first string is a node label
        
        :param string: the string
        :param bb: the bb
        :return: the new string
        :rtype: string
    """
    match = re.search(r'<bb\\\ \d+\\>', string)
    
    if match == None:
        return string
    match_bb = int(re.search(r'\d+', match.group()).group())
    
    if match_bb > bb:
        string = string.replace('<bb\ ' + str(match_bb), '<bb\ ' + str(match_bb + 1))
    return string[0:match.end()] + update_bb_string(string[match.end():], bb)

In [1655]:
print(node.get_attributes()["label"])
print("----------")
print(update_bb_string(node.get_attributes()["label"], 0))

"{ FREQ:0 |\<bb\ 3\>:\l\
|_1\ =\ foo\ (b);\l\
|if\ (_1\ \>\ 3)\l\
\ \ goto\ \<bb\ 4\>;\ [0.00%]\l\
else\l\
\ \ goto\ \<bb\ 5\>;\ [0.00%]\l\
}"
----------
"{ FREQ:0 |\<bb\ 4\>:\l\
|_1\ =\ foo\ (b);\l\
|if\ (_1\ \>\ 3)\l\
\ \ goto\ \<bb\ 5\>;\ [0.00%]\l\
else\l\
\ \ goto\ \<bb\ 6\>;\ [0.00%]\l\
}"


In [1656]:
def update_node_name(node_name, bb):
    """
        Update the node_name if its bb is superior to the bb given
        
        :param node_name: the name to update
        :param bb: the bb threshold
        :return: the node_name (updated)
        :rtype: String
    """
    node_number = int(re.search(r'\d+', re.search(r'block_\d+', node_name).group()).group())
    if bb < node_number:
        return node_name.replace('block_' + str(node_number), 'block_' + str(node_number+1))
    return node_name

In [1657]:
print(node.get_name())
print("----------")
print(update_node_name(node.get_name(), 0))

fn_1_basic_block_3
----------
fn_1_basic_block_4


In [1658]:
def update_nodes(nodes, bb):
    """
        Updates all the nodes whose bb are greater or equal to the bb given.
        
        :param nodes: the nodes to update
        :param bb: the bb
    """
    
    for node in nodes:
        node.set("label", update_bb_string(node.get_attributes()["label"], bb))
        node.set_name(update_node_name(node.get_name(), bb))


In [1659]:
print(node.get_attributes()["label"] + "\n" + node.get_name())
print("----------")
update_nodes([node], bb)
print(node.get_attributes()["label"] + "\n" + node.get_name())

"{ FREQ:0 |\<bb\ 3\>:\l\
|_1\ =\ foo\ (b);\l\
|if\ (_1\ \>\ 3)\l\
\ \ goto\ \<bb\ 4\>;\ [0.00%]\l\
else\l\
\ \ goto\ \<bb\ 5\>;\ [0.00%]\l\
}"
fn_1_basic_block_3
----------
"{ FREQ:0 |\<bb\ 3\>:\l\
|_1\ =\ foo\ (b);\l\
|if\ (_1\ \>\ 3)\l\
\ \ goto\ \<bb\ 5\>;\ [0.00%]\l\
else\l\
\ \ goto\ \<bb\ 6\>;\ [0.00%]\l\
}"
fn_1_basic_block_3


#### On veut récupérer les noeuds qui pourront être update ce qui veut dire qu'il faut itérer dans les sous-graphes enfants et parents

In [1660]:
def get_nodes_to_update(subgraph, graph_name, starter=0):
    """
        Retrive all the nodes to update from the parents to the children.
        
        :param subgraph: the current subgraph
        :param graph_name: the name of the global graph
        :param starter: the flag, 0 if you are the subgraph starter, 1 else
        :return: the nodes to update
        :rtype: list
    """
    global subgraphs, parents
    
    nodes = extract_nodes(subgraph)
    
    for child_subgraph in subgraph.get_subgraph_list():
        nodes += get_nodes_to_update(child_subgraph, graph_name, 1)
    
    parent = parents[subgraph.get_name()].get_name()
    while(parent != graph_name and starter == 0):
        nodes += extract_nodes(parents[subgraph.get_name()])
        parent = parents[parent].get_name()
    return nodes

In [1661]:
nodes_while = get_nodes_to_update(subgraphs["1_1"], G.get_name())
string_while = ""
for node_while in nodes_while:
    string_while += node_while.get_name() + " "
nodes_main = get_nodes_to_update(subgraphs["main"], G.get_name())
string_main=""
for node_main in nodes_main:
    string_main += node_main.get_name() + " "
print(string_while + "\n")
print(string_main)

fn_1_basic_block_3 fn_1_basic_block_4 fn_1_basic_block_5 fn_1_basic_block_6 fn_1_basic_block_0 fn_1_basic_block_1 fn_1_basic_block_2 fn_1_basic_block_7 fn_1_basic_block_8 

fn_1_basic_block_0 fn_1_basic_block_1 fn_1_basic_block_2 fn_1_basic_block_7 fn_1_basic_block_8 fn_1_basic_block_3 fn_1_basic_block_4 fn_1_basic_block_5 fn_1_basic_block_6 


#### Create the new node

In [1662]:
def create_new_node(subgraph, prev_node, label, bb):
    """
        Create a new node
        
        :param subgraph: the subgraph where lives the new node
        :param prev_node: the node that necessits a child
        :param label: the label of the new node
        :param bb: the bb of the previous node
        :return: the new node
        :rtype: Node
    """
    return add_node(subgraph, update_node_name(prev_node.get_name(), bb-1), label=update_bb_string(label, bb-1))

In [1663]:
print(node.get_name())
new_node = create_new_node(subgraphs["1_1"], node, label_node2, 3)
print(node.get_name())
print(new_node.get_name())
print(new_node.get_label())

fn_1_basic_block_3
fn_1_basic_block_3
fn_1_basic_block_4
"{ FREQ:0 |\<bb\ 4\>:\l\
|_1\ =\ return\ foo\ (b);\l\
|if\ (_1\ \>\ 3)\l\
\ \ goto\ \<bb\ 5\>;\ [0.00%]\l\
else\l\
\ \ goto\ \<bb\ 6\>;\ [0.00%]\l\
}"


# Create the edges

In [1664]:
def get_top_parent(subgraph, graph_name):
    """
        Get the top parent in the arborescence of the graph after the root (graph_name)
        
        :param subgraph: the subgraph to find the parent
        :param graph_name: the root of the arborescence
        :return: The top parent of the subgraph
        :rtype: Subgraph
    """
    global parents
    
    if(subgraph.get_name() == graph_name):
        return None
    
    current = subgraph
    while(parents[current.get_name()].get_name() != graph_name):
        current = parents[current.get_name()]
    return current
    

In [1665]:
def get_bb(node_name):
    """
        Get the bb of a node_name
        
        :param node_name: the string to find the bb
        :return: the bb
        :rtype: int
    """
    return int(re.search(r'\d+', re.search(r'block_\d+', node_name).group()).group())

In [1666]:
def update_edge_node_name(node_name, node_number):
    """
        Update the node_name coming from an edge based on the node_number
        
        :param node_name: the node name
        :param node_number: the number to update
        :return: The new node_name
        :rype: String
    """
    return node_name.replace('block_' + str(node_number), 'block_' + str(node_number+1))

In [1667]:
def update_edges(subgraph, graph_name, bb):
    """
        Update the edges of the top parent of the subgraph
        
        :param subgraph: the subgraph which top parent's edges has to be updated
        :param graph_name: the root of the arborescence of subgraph
        :param bb: the bb from which it updates the edges
    """
    top_subgraph = get_top_parent(subgraph, graph_name)
    edges = extract_edges(top_subgraph)
    for edge in edges:
        if(edge.get_style() is not None):
            style = edge.get_style()
        if(edge.get_color() is not None):
            color = edge.get_color()
        if(edge.get_label() is not None):
            label = edge.get_label()
        node_head = edge.get_source()
        node_tail = edge.get_destination()
        bb_head = get_bb(node_head)
        bb_tail = get_bb(node_tail)
        if(bb_head >= bb or bb_tail > bb):
            top_subgraph.del_edge(node_head, node_tail, 0)
            if bb_head >= bb:
                if bb_tail > bb:
                    add_edge(top_subgraph, update_edge_node_name(node_head, bb_head), update_edge_node_name
                             (node_tail, bb_tail), style=style, color=color, label=label)
                else:
                    add_edge(top_subgraph, update_edge_node_name(node_head, bb_head), node_tail, style=style, 
                             color=color, label=label)
            else:
                add_edge(top_subgraph, node_head, update_edge_node_name(node_tail, bb_tail), 
                         style=style, color=color, label=label)
                
            #si bb_n < bb et bb_s <= bb on touche pas
            #sinon
            #    si bb_n >= bb:
            #        si bb_s >= bb:
            #            creer edge (n+1, s+1)
            #        sinon:
            #            creer edge (n+1, s)
            #    sinon:
            #        si bb_s > bb:
            #          creer edge (n, s+1)  

In [1668]:
update_edges(subgraphs["main"], parents[subgraphs["main"].get_name()].get_name(), 3)

In [1669]:
def create_new_edge(graph, node_name, subgraph_function):
    """
        Create the edges corresponding of the call and return of the function
        
        :param graph: edges are added in the graph
        :param node_name: the node from where the call is made
        :param subgraph_function: the subgraph of the function called
    """
    nodes_function = extract_nodes(subgraph_function)
    add_edge(graph, node_name+":s", nodes_function[0].get_name()+":n", color="green")
    add_edge(graph, nodes_function[1].get_name()+":s", update_edge_node_name(node_name, get_bb(node_name))+":n", color="red")
    return
    

# The export

#### For all the subgraph, we need to update their names because there is a syntax error during the output otherwise

In [1670]:
def recreate_subgraphs_name():
    """
        Recreate the name of the subgraphs to write them in dot format
    """
    global subgraphs
    for (name, subgraph) in subgraphs.items():
        subgraph.set_name("\"cluster_" + subgraph.get_name() + "\"")

In [1671]:
recreate_subgraphs_name()
print(G.get_subgraph_list()[0].get_name())

"cluster_foo"


#### Export the graph

In [1672]:
def export_graph(graph, name_file, format_export):
    """
        Export the graph with its name and its format given
        
        :param graph: the graph
        :param name_file: the name of the file
        :param format_export: the format of the export
    """
    im_name = ('{}.' + format_export).format('./' + name_file)
    if (format_export == "png"):
        graph.write_png(im_name)
    elif (format_export ==  "dot"):
        graph.write_dot(im_name)
    else:
        raise LookupError

In [1673]:
export_graph(G, "test", "png")
export_graph(G, "test", "dot")

In [1674]:
def main(file):
    """
        Take a graph from the file and format it to adopt our constraints
        
        :param file: the file to get
        :return: the graph
        :rtype: Graph
    """
    global subgraphs, parents
    graph = graph_from_dot(file)
    subgraphs = {}
    parents = {}
    extract_subgraphs([graph])
    
    for (name, subgraph) in subgraphs.items():
        nodes = extract_nodes(subgraph)
        for node in nodes:
            (name_function, result, function_call_line) = analyse_label_function_calls(node)
            if name_function is not None:
                (label_node1, label_node2, bb) = create_labels(node, result, function_call_line)
                node.set_label(label_node1)
                nodes_to_update = get_nodes_to_update(subgraph, graph.get_name())
                update_nodes(nodes_to_update, bb)
                nodes.append(create_new_node(subgraph, node, label_node2, bb))
                update_edges(subgraph, graph.get_name(), bb)
                create_new_edge(graph, node.get_name(), subgraphs[name_function])
    recreate_subgraphs_name()
    export_graph(graph, "main_output", "png")
    export_graph(graph, "main_output", "dot")
    return graph

In [1675]:
graph = main("../Tests/test_plusieurs_appels_imbriqués/test.c.011t.cfg.dot")

# The result

<img src="main_output.png" style="width:400px">